In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

# Ensure your SerperDev API key is set in the environment variables
os.environ["SERPER_API_KEY"] = os.environ["SERPER_DEV_API_KEY"]

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'

In [2]:
from crewai import Agent, Task, Crew
from crewai_tools import PDFSearchTool


/Users/ashish/Desktop/vettura-genai/Codes/genai_env/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [17]:

# Initialize the PDFSearchTool with the path to your resume
pdf_tool = PDFSearchTool(pdf='Week_4/Day_3/Resume_Sample.pdf')


# Define the agent with role, goal, and backstory
resume_agent = Agent(
    name="Resume Summarizer",
    role="Resume Analysis Expert",
    goal="Efficiently extract and summarize key information from PDF resumes to assist in candidate evaluation.",
    backstory=(
        "As a seasoned professional in resume analysis, you have a keen eye for identifying pertinent details "
        "that highlight a candidate's qualifications. Your expertise ensures that no critical information is overlooked, "
        "providing comprehensive summaries that aid in the recruitment process."
    ),
    tools=[pdf_tool],
    verbose=True,
)

# Define the task
resume_task = Task(
    description="Read and summarize a given PDF resume.",
    agent=resume_agent,
    expected_output="A structured summary of key resume details."
)


In [ ]:
# Create the crew
resume_crew = Crew(agents=[resume_agent], tasks=[resume_task])

# Execute the task
candidate_summary = resume_crew.kickoff()
print(candidate_summary)

In [4]:
from crewai_tools import ScrapeWebsiteTool

# Initialize the tool with the website URL
scrape_website_tool = ScrapeWebsiteTool(website_url='https://www.linkedin.com/jobs/view/4147112954')

In [5]:
# Define the agent
job_agent = Agent(
    name="Job Requirement Summarizer",
    role="Extracts and summarizes job details from a specified website.",
    goal="Efficiently extract and summarize key information from job postings to assist in candidate evaluation.",
    backstory=(
        "As an expert in web data extraction, you specialize in retrieving and summarizing job postings "
        "to provide concise and relevant information for recruitment purposes."
    ),
    tools=[scrape_website_tool],
    Verbose=True
)

# Define the task
job_task = Task(
    description="Scrape and summarize job details from the linkedin job posting.",
    agent=job_agent,
    expected_output="A structured summary of key job details."
)

In [ ]:

# Create the crew
job_discovery_crew = Crew(agents=[job_agent], tasks=[job_task])

# Execute the task
job_summary = job_discovery_crew.kickoff()
print(job_summary)

In [6]:
from crewai_tools import SerperDevTool
from crewai import Process
search_tool = SerperDevTool()

In [7]:
# Define the Company Research Agent
research_agent = Agent(
    name="Company Research Agent",
    role="Conducts comprehensive web searches to gather detailed information about target companies.",
    goal="Provide in-depth insights into the company where the candidate is applying for job identified by job_task. Include culture, values, and expectations.",
    backstory=(
        "As a seasoned corporate analyst with years of experience in market research, "
        "you possess a keen ability to uncover and synthesize pertinent information about organizations. "
        "Your expertise enables you to provide valuable insights that help candidates align their applications "
        "with the company's culture and expectations."
    ),
    tools=[search_tool],
    verbose=True
)

# Define the research task
research_task = Task(
    description="Investigate and summarize the company's culture, values, and the temperament they seek in candidates for the specified role.",
    agent=research_agent,
    context=[job_task],  # Ensure job_task is defined prior to this
    expected_output="A structured summary detailing the company's profile, job requirements, and the desired candidate temperament."
)

In [ ]:
# Create the crew
company_research_crew = Crew(agents=[job_agent, research_agent], 
                            tasks=[job_task, research_task], 
                            process=Process.sequential) 

# Execute the task
market_research = company_research_crew.kickoff()
print(market_research)

In [8]:
from crewai_tools import FileWriterTool
file_writer = FileWriterTool() 
resume_refiner = Agent(
    name="Resume Refiner",
    role="Refines the resume to allign with the job requirement",
    goal="Modify the resume analyzed by the resume_agent to align with the job requirements identified by the research_agent",
    backstory="A professional resume writer skilled in tailoring resumes for specific roles.",
    tools=[file_writer],
    verbose=True
)

refine_resume_task = Task(
    description="Update the resume to align with the job requirements.",
    agent=resume_refiner,
    context= [research_task, research_task],
    expected_output="A refined resume tailored to the job description in markdown format. Write the resume to local file names refined_resume.md"
)

In [ ]:
# Create the crew
resume_refinement_crew = Crew(
    agents=[resume_agent, job_agent, research_agent, resume_refiner],
    tasks=[resume_task, job_task, research_task, refine_resume_task],
    process=Process.sequential
)

resume_refinement_crew.kickoff()

In [9]:
# Define the Cover Letter Writer agent with refined attributes
Cover_Letter_Writer = Agent(
    name="Cover Letter Writer",
    role="Crafts personalized cover letters tailored to specific job applications.",
    goal="Compose compelling cover letters that align the candidate's qualifications with the job requirements identified by the research_agent.",
    backstory=(
        "As an experienced career consultant and professional writer, you specialize in creating persuasive cover letters "
        "that effectively highlight a candidate's strengths and align them with the targeted job opportunities."
    ),
    tools=[file_writer],
    verbose=True
)

cover_letter_writing_task = Task(
    description="Write a cover letter for the job.",
    agent=Cover_Letter_Writer,
    context=[research_task, refine_resume_task],
    expected_output="A well crafted cover letter tailored to the job description and refined resume in markdown format. Write the letter to local file as Cover_Letter.md"
)

# Define the Salary Negotiation Strategist agent
Salary_Negotiation_Bot = Agent(
    name="Salary Negotiation Strategist",
    role="Analyzes job requirements and candidate qualifications to develop effective salary negotiation strategies.",
    goal="Equip the candidate with personalized negotiation tactics and realistic salary expectations based on their resume and the target job.",
    backstory=(
        "As a seasoned career advisor with extensive experience in compensation analysis and negotiation tactics, "
        "you specialize in guiding professionals to secure optimal compensation packages by aligning their unique skills and experiences "
        "with current market trends and specific job demands."
    ),
    tools=[file_writer],
    verbose=True
)

# Define the salary negotiation task
salary_negotiation_task = Task(
    description="Analyze the candidate's resume and the specified job requirements to identify current salary trends and develop tailored negotiation strategies.",
    agent=Salary_Negotiation_Bot,
    context=[research_task, refine_resume_task],  # Ensure these tasks are defined and provide necessary context
    expected_output=(
        "A comprehensive document in markdown format titled 'Salary_Negotiation_Strategy.md' containing:\n"
        "- An analysis of current salary trends for the specified job role.\n"
        "- A personalized salary negotiation strategy for the candidate, considering their resume and the job requirements.\n"
        "- Recommended salary range expectations based on market data and the candidate's qualifications."
    )
)

# Define the Interview Preparation Agent
Interview_Prep_Bot = Agent(
    name="Interview Trainer",
    role="Prepares the candidate for job interviews by providing tailored strategies and mock interview questions.",
    goal="Enhance the candidate's confidence and preparedness by simulating interview scenarios, providing strategic answers, and crafting insightful questions.",
    backstory=(
        "As a former HR specialist and career coach, you have trained countless professionals in acing job interviews. "
        "Your expertise lies in understanding company expectations, behavioral and technical interview patterns, "
        "and guiding candidates through structured mock interview sessions. "
        "You use real-world data and industry insights to formulate the best possible interview strategies."
    ),
    tools=[file_writer],
    verbose=True
)

# Define the interview training task
interview_training_task = Task(
    description="Analyze the candidate's refined resume and company research data to generate an interview preparation plan. "
                "Create a strategy document with expected interview questions (Both technical and hr rounds) and suggested answers, focusing on company culture, job role, "
                "and candidate strengths.",
    agent=Interview_Prep_Bot,
    context=[research_task, refine_resume_task],
    expected_output=(
        "Two structured documents:\n"
        "1. 'Interview_Strategy.md' - A step-by-step interview preparation strategy including behavioral techniques, common pitfalls, "
        "and best practices tailored for the specific company.\n"
        "2. 'Interview_Questions.md' - A list of expected interview questions along with suggested answers based on the job description, "
        "company expectations, and candidate's resume. Creaate all files in markdown format."
    )
)

'\n# Define the manager agen\nmanager_agent = Agent(\n    role="Project Manager",\n    goal="Efficiently manage the crew and ensure high-quality task completion",\n    backstory="You\'re an experienced project manager, skilled in overseeing complex projects and guiding teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard.",\n    allow_delegation=True,\n    verbose=True\n)\n\n# Define the manager task\nmanager_task = Task(\n    description="Oversee the job application preparation process, ensuring each agent completes their tasks effectively."\n    "Workflow:\n"\n        "1. Summarize the candidate\'s resume to identify key strengths and improvement areas. You may delegate to Resume Summarizer\n"\n        "2. Analyze the job description through delegation to Job Requirement Summarizer.\n"\n        "3. Conduct thorough company research to understand expectations by delegating to agent named 

In [10]:

# Create the crew
Job_Application_Crew = Crew(
    agents=[resume_agent, job_agent, research_agent, resume_refiner, Cover_Letter_Writer, Salary_Negotiation_Bot, Interview_Prep_Bot],
    tasks=[resume_task, job_task, research_task, refine_resume_task, cover_letter_writing_task, salary_negotiation_task, interview_training_task],
    process=Process.sequential,
)

job_application = Job_Application_Crew.kickoff()
print(job_application)

2025-02-14 23:10:42,460 - 7955664896 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Resume Analysis Expert
## Task: Read and summarize a given PDF resume.


# Agent: Resume Analysis Expert
## Thought: I need to extract the content from the PDF resume to analyze it thoroughly.
## Using tool: Search a PDF's content
## Tool Input: 
"{\"query\": \"content\"}"
## Tool Output: 
Relevant Content:
Levanssh Kansal ♂¶obile+91.9530526601 /envel⌢pelevansshkansal01@gmail.com /linkedinLevanssh-Kansal /githubLevanssh Education Indian Institute of Information Technology Una Una, Himachal Pradesh B.Tech. in Electronics and Communication Engineering (CGPA: 7.5 / 10.00) 2022-2026 SSD Krishna Vatika School, Bathinda Bathinda, Punjab Class XII PCM (92%). 2020-2022 Projects Connexus | Full-Stack Web Development, HTML, CSS, Javascript, NodeJS, MySQL [GitHub Link] • A web application connecting influencers with business owners for product and service promotion. • Designed dashboards for influencers and brand owners, with advanced search filters. • Delivered a user-engaging interface

In [18]:

# Define the manager agent
manager_agent = Agent(
    role="Head of Counselling",
    goal="A coordinated execution of all tasks by delegating to the crew members leading to a generate a comprehensive job application package.",
    backstory="You're an experienced job counsellor leading an agency that helps in preparing the candidates for getting their dream job.",
    allow_delegation=True,
    verbose=True,
    llm = "gpt-4o"
)

# Define the manager task
manager_task = Task(
    description="Oversee the job application preparation process, ensuring each agent completes their tasks effectively."
    "Workflow:\n"
        "1. Summarize the candidate's resume to identify key strengths and improvement areas. You may delegate to Resume Summarizer agent\n"
        "2. Analyze the job description through delegation to Job Requirement Summarizer agent.\n"
        "3. Conduct thorough company research to understand expectations by delegating to agent named Company Research Agent.\n"
        "4. Align the candidate’s resume to the job role and company culture, ensuring an optimized application. You may delegate to Resume Refiner\n"
        "5. Generate a compelling cover letter to complement the refined resume. You may delegate to Cover Letter Writer\n"
        "6. Develop a salary negotiation strategy based on industry trends, job role, and candidate’s experience. You may delegate to Salary Negotiation Strategist.\n"
        "7. Prepare an in-depth interview strategy, including expected questions and recommended answers, to enhance the candidate’s confidence. You may delegate to Interview Trainer.\n",
    agent=manager_agent,
    expected_output="A coordinated execution of all tasks by delegating to the crew members leading to a generate a comprehensive job application package."
)

# Create the crew
Job_Application_Crew = Crew(
    agents=[resume_agent, job_agent, research_agent, resume_refiner, Cover_Letter_Writer, Salary_Negotiation_Bot, Interview_Prep_Bot],
    tasks=[manager_task, resume_task, job_task, research_task, refine_resume_task, cover_letter_writing_task, salary_negotiation_task, interview_training_task],
    process=Process.hierarchical,
    manager_agent=manager_agent,
    memory=True,
    manager_llm= "gpt-4o"
)

job_application = Job_Application_Crew.kickoff()
print(job_application)

2025-02-14 23:31:24,862 - 7955664896 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed
2025-02-14 23:31:24,864 - 7955664896 - rag_storage.py-rag_storage:96 - WARNING: Trimming file name from 514 to 255 characters.
2025-02-14 23:31:24,979 - 7955664896 - rag_storage.py-rag_storage:96 - WARNING: Trimming file name from 514 to 255 characters.


# Agent: Head of Counselling
## Task: Oversee the job application preparation process, ensuring each agent completes their tasks effectively.Workflow:
1. Summarize the candidate's resume to identify key strengths and improvement areas. You may delegate to Resume Summarizer agent
2. Analyze the job description through delegation to Job Requirement Summarizer agent.
3. Conduct thorough company research to understand expectations by delegating to agent named Company Research Agent.
4. Align the candidate’s resume to the job role and company culture, ensuring an optimized application. You may delegate to Resume Refiner
5. Generate a compelling cover letter to complement the refined resume. You may delegate to Cover Letter Writer
6. Develop a salary negotiation strategy based on industry trends, job role, and candidate’s experience. You may delegate to Salary Negotiation Strategist.
7. Prepare an in-depth interview strategy, including expected questions and recommended answers, to enhance t

KeyboardInterrupt: 